In [3]:
import pandas as pd
import numpy as np

# 1. Load data


In [4]:
data_path = "ChallengeXHEC23022024.xlsx"
dict_df_sheets = pd.read_excel(data_path, sheet_name=None)
df_hist_jan = dict_df_sheets["JAN24"].copy()
df_clients = dict_df_sheets["clients"].copy()
df_inter = dict_df_sheets["intervenants"].copy()

# 2. Data exploration preprocessing


In [5]:
# df_hist_jan.rename(
#     {
#         "ID Client": "id_client",
#         "ID Intervenant": "id_inter",
#         "Date": "date",
#         "Heure de début": "start",
#         "Heure de fin": "end",
#         "Prestation": "obj",
#     },
#     axis=1,
#     inplace=True,
# )

# df_clients.rename(
#     {"ID Client": "id_client", "Latitude": "lat_client", "Longitude": "lon_client"},
#     axis=1,
#     inplace=True,
# )

# df_inter.rename(
#     {
#         "ID Intervenant": "id_inter",
#         "Latitude": "lat_inter",
#         "Longitude": "lon_inter",
#         "Compétences": "skills",
#         "Permis": "license",
#         "Véhicule personnel": "car",
#         "Dispo / Indispo": "free",
#     },
#     axis=1,
#     inplace=True,
# )

In [6]:
# df_hist_jan["date"] = pd.to_datetime(df_hist_jan["date"])
# df_hist_jan["start"] = pd.to_datetime(df_hist_jan["date"])
# df_hist_jan["date"] = pd.to_datetime(df_hist_jan["date"])

# 5. Modeling with Google OR Tools


In [7]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from functools import partial

### 5.C.1. With initial locations, time windows, bike/car time travel, task execution time and skills with REAL DATA


In [8]:
c_c_dist = pd.read_csv("client_client_distance.csv")
c_l_need = pd.read_csv("client_loc_need.csv")
c_n_dist = pd.read_csv("client_nurse_distance.csv")

list_clients_1 = c_c_dist["ID Client 1"].unique()
list_clients_2 = c_c_dist["ID Client 2"].unique()
full_client_list = list(set([*list_clients_1, *list_clients_2]))
nb_full_client = len(full_client_list)

c_l_need.drop(columns=["Unnamed: 0", "Latitude", "Longitude"], inplace=True)

In [9]:
client_id_to_idx = {}
client_idx_to_id = {}
for k in range(len(full_client_list)):
    client_id_to_idx[full_client_list[k]] = k
    client_idx_to_id[k] = full_client_list[k]

In [10]:
def convert_duration(str_duration):
    if "hour" in str_duration:
        list = str_duration.split()
        nb_minutes = int(list[0]) * 60 + int(list[2])
        return nb_minutes
    else:
        list = str_duration.split()
        nb_minutes = int(list[0])
        return nb_minutes

In [11]:
def define_client_time_matrix(
    df, vehicle_type, client_id_to_idx=client_id_to_idx, out_value=4000
):
    list_clients_1 = df["ID Client 1"].unique()
    list_clients_2 = df["ID Client 2"].unique()
    full_list = list(set([*list_clients_1, *list_clients_2]))

    matrix = [[0 for i in range(len(full_list))] for k in range(len(full_list))]
    col_type = "Duration_" + vehicle_type
    for client_1_id in full_list:
        client_1_idx = client_id_to_idx[client_1_id]
        for client_2_id in full_list:
            client_2_idx = client_id_to_idx[client_2_id]
            if client_1_id != client_2_id:
                try:
                    travel_time = convert_duration(
                        df.loc[
                            (df["ID Client 1"] == client_1_id)
                            & (df["ID Client 2"] == client_2_id),
                            col_type,
                        ].values[0]
                    )
                    matrix[client_1_idx][client_2_idx] = travel_time
                except IndexError:
                    try:
                        travel_time = convert_duration(
                            df.loc[
                                (df["ID Client 1"] == client_2_id)
                                & (df["ID Client 2"] == client_1_id),
                                col_type,
                            ].values[0]
                        )
                        matrix[client_1_idx][client_2_idx] = travel_time
                    except IndexError:
                        matrix[client_1_idx][client_2_idx] = out_value

    return np.array(matrix)

In [12]:
client_time_matrix_car = define_client_time_matrix(c_c_dist, vehicle_type="Car")
client_time_matrix_bike = define_client_time_matrix(c_c_dist, vehicle_type="Bike")

In [13]:
list_inter_columns = c_n_dist.columns[2:]
full_inter_list = list(set([int(x.split(sep="_")[2]) for x in list_inter_columns]))

inter_id_to_idx = {}
inter_idx_to_id = {}
for k in range(len(full_inter_list)):
    inter_id_to_idx[full_inter_list[k]] = k
    inter_idx_to_id[k] = full_inter_list[k]

In [14]:
def define_inter_time_matrix(
    df,
    full_client_list,
    inter_id_to_idx=inter_id_to_idx,
    client_id_to_idx=client_id_to_idx,
    vehicle_type="Car",
    out_value=4000,
):
    list_inter_columns = df.columns[2:]
    full_inter_list = list(set([int(x.split(sep="_")[2]) for x in list_inter_columns]))

    matrix = [
        [0 for i in range(len(full_client_list))] for k in range(len(full_inter_list))
    ]

    for inter_id in full_inter_list:
        vehicle_str = "_Driving" if vehicle_type == "Car" else "_Bicycling"
        col_name = "Duration_Intervenant_" + str(inter_id) + vehicle_str
        inter_idx = inter_id_to_idx[inter_id]

        for client_id in full_client_list:
            client_idx = client_id_to_idx[client_id]
            try:
                travel_time = convert_duration(
                    df.loc[df["ID Client"] == client_id, col_name].values[0]
                )
                matrix[inter_idx][client_idx] = travel_time
            except IndexError:
                matrix[inter_idx][client_idx] = out_value

    return np.array(matrix)

In [15]:
inter_time_matrix_car = define_inter_time_matrix(
    c_n_dist, full_client_list, vehicle_type="Car"
)
inter_time_matrix_bike = define_inter_time_matrix(
    c_n_dist, full_client_list, vehicle_type="Bike"
)

In [16]:
def create_full_time_matrix(client_time_matrix, inter_time_matrix, out_value=4000):
    nb_client = client_time_matrix.shape[0]
    nb_inter = inter_time_matrix.shape[0]

    matrix = [
        [out_value for i in range(nb_client + nb_inter)]
        for k in range(nb_client + nb_inter)
    ]

    for i in range(nb_client):
        for j in range(nb_client):
            matrix[i][j] = client_time_matrix[i][j]

    for i in range(nb_inter):
        for j in range(nb_client):
            matrix[i + nb_client][j] = inter_time_matrix[i][j]
            matrix[j][i + nb_client] = matrix[i + nb_client][j]

    return np.array(matrix)

In [17]:
full_time_matrix_car = create_full_time_matrix(
    client_time_matrix_car, inter_time_matrix_car
)
full_time_matrix_bike = create_full_time_matrix(
    client_time_matrix_bike, inter_time_matrix_bike
)

In [18]:
list_skills = c_l_need["Prestation"].unique()

condition = c_l_need["Date"] == "2024-01-01"
c_l_test = c_l_need[condition]
c_l_test.drop(columns=["Date"], inplace=True)
# c_l_test = c_l_test[:70]
c_l_test.head()

/var/folders/95/2w4zxm7j571dt7gznpftznw80000gn/T/ipykernel_42525/3667956813.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_l_test.drop(columns=["Date"], inplace=True)


,ID Client,Prestation,Durée,tranche_horaire
0,559475456,REPAS,30.0,"[7, 9]"
1,559277088,TOILETTE,45.0,"[7, 11]"
2,87852633,TOILETTE,45.0,"[7, 11]"
3,243033408,TOILETTE,30.0,"[7, 11]"
4,814940942,TOILETTE,95.0,"[7, 11]"


# 6. Data preparation for scenarios


In [19]:
data_path = "ChallengeXHEC23022024.xlsx"
dict_df_sheets = pd.read_excel(data_path, sheet_name=None)
df_hist_jan = dict_df_sheets["JAN24"].copy()
df_clients = dict_df_sheets["clients"].copy()
df_inter = dict_df_sheets["intervenants"].copy()

In [20]:
list_jan_dates = ["2024-01-" + ("0" if k < 10 else "") + str(k) for k in range(1, 32)]
df_inter_columns = [*df_inter.columns[:7], *list_jan_dates]
df_inter.columns = df_inter_columns

In [21]:
client_locations = [
    (df_clients["Latitude"][k], df_clients["Longitude"][k])
    for k in range(len(df_clients))
]

inter_locations = [
    (df_inter["Latitude"][k], df_inter["Longitude"][k]) for k in range(len(df_inter))
]

In [22]:
client_id_to_location = {}
for k in range(len(df_clients)):
    client_id_to_location[int(df_clients["ID Client"][k])] = client_locations[k]

inter_id_to_location = {}
for k in range(len(df_inter)):
    inter_id_to_location[int(df_inter["ID Intervenant"][k])] = inter_locations[k]

In [23]:
def get_list_og_available(date, df_inter):
    condition = df_inter[date] == 1
    list_available_og = list(df_inter[condition]["ID Intervenant"])
    return list_available_og

# 6.0. Travel time estimator


In [24]:
from sklearn.ensemble import RandomForestRegressor

In [25]:
nb_clients = len(full_client_list)
nb_inter = len(full_inter_list)
nb_loc = len(full_time_matrix_car)

travel_time_data = []

for i in range(nb_loc - 1):
    for j in range(i + 1, nb_loc):
        row = []
        if i < nb_clients:
            client_id_1 = client_idx_to_id[i]
            row.append(client_id_to_location[client_id_1][0])
            row.append(client_id_to_location[client_id_1][1])

            if j < nb_clients:
                client_id_2 = client_idx_to_id[j]
                row.append(client_id_to_location[client_id_2][0])
                row.append(client_id_to_location[client_id_2][1])
            else:
                inter_id_2 = inter_idx_to_id[j - nb_clients]
                row.append(inter_id_to_location[inter_id_2][0])
                row.append(inter_id_to_location[inter_id_2][1])

        else:
            inter_id = inter_idx_to_id[i - nb_clients]
            row.append(inter_id_to_location[inter_id][0])
            row.append(inter_id_to_location[inter_id][1])

            if j < nb_clients:
                client_id_2 = client_idx_to_id[j]
                row.append(client_id_to_location[client_id_2][0])
                row.append(client_id_to_location[client_id_2][1])
            else:
                inter_id_2 = inter_idx_to_id[j - nb_clients]
                row.append(inter_id_to_location[inter_id_2][0])
                row.append(inter_id_to_location[inter_id_2][1])

        row.append(full_time_matrix_car[i][j])
        row.append(full_time_matrix_bike[i][j])

        travel_time_data.append(row)

travel_time_df = pd.DataFrame(
    travel_time_data, columns=["x_1", "y_1", "x_2", "y_2", "car", "bike"]
)
travel_time_df

,x_1,y_1,x_2,y_2,car,bike
0,48.732218,1.363744,48.705077,1.333307,7,14
1,48.732218,1.363744,48.638658,1.517034,24,63
2,48.732218,1.363744,48.787278,1.441202,16,38
3,48.732218,1.363744,48.768307,1.403410,10,24
4,48.732218,1.363744,48.725093,1.427367,11,25
...,...,...,...,...,...,...
9586,48.758099,1.210611,48.689927,1.351054,4000,4000
9587,48.758099,1.210611,48.763226,1.241120,4000,4000
9588,48.729206,1.371985,48.689927,1.351054,4000,4000
9589,48.729206,1.371985,48.763226,1.241120,4000,4000


In [26]:
condition = travel_time_df["car"] != 4000
car_travel_time_df = travel_time_df[condition].copy()
X_car = np.array(car_travel_time_df.drop(columns=["car", "bike"]))
y_car = np.array(car_travel_time_df["car"])

car_time_travel_estimator = RandomForestRegressor()
car_time_travel_estimator.fit(X_car, y_car)

RandomForestRegressor()

In [27]:
condition = travel_time_df["bike"] != 4000
bike_travel_time_df = travel_time_df[condition].copy()
X_bike = np.array(bike_travel_time_df.drop(columns=["car", "bike"]))
y_bike = np.array(bike_travel_time_df["bike"])

bike_time_travel_estimator = RandomForestRegressor()
bike_time_travel_estimator.fit(X_bike, y_bike)

RandomForestRegressor()

## 6.1. Add new client


The goal is to create artificially a new client the following way:

- Get a random ('Prestation', 'Durée', 'Tranche horaire') from an existing client;
- Get a random location

From the random location, infere the travel time to each other node


In [28]:
def add_clients_in_time_matrix(
    client_synthetic_locations,
    time_matrix_car,
    time_matrix_bike,
    df_og_need,
    list_available_inter,
):
    nb_og_clients = len(df_og_need)
    len_new_matrix = (
        nb_og_clients + len(client_synthetic_locations) + len(list_available_inter)
    )
    new_car_matrix = [
        [4000 for k in range(len_new_matrix)] for k in range(len_new_matrix)
    ]
    new_bike_matrix = [
        [4000 for k in range(len_new_matrix)] for k in range(len_new_matrix)
    ]

    nb_synth_clients = len(client_synthetic_locations)

    for i in range(len_new_matrix - 1):
        for j in range(i + 1, len_new_matrix):
            if i < nb_og_clients:
                if j < nb_og_clients:
                    new_car_matrix[i][j] = time_matrix_car[i][j]
                    new_bike_matrix[i][j] = time_matrix_bike[i][j]
                elif nb_og_clients <= j < nb_og_clients + nb_synth_clients:
                    # client_id_1 = client_idx_to_id[i]
                    client_id_1 = df_og_need["ID Client"][i]
                    client_loc_1 = client_id_to_location[client_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                else:
                    new_car_matrix[i][j] = time_matrix_car[i][j - nb_synth_clients]
                    new_bike_matrix[i][j] = time_matrix_bike[i][j - nb_synth_clients]
            elif nb_og_clients <= i < nb_og_clients + nb_synth_clients:
                if j < nb_og_clients + nb_synth_clients:
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                else:
                    # inter_id_1 = inter_idx_to_id[j - nb_og_clients - nb_synth_clients]
                    # print(i)
                    # print(j)
                    # print(j - nb_og_clients - nb_synth_clients)
                    # print("\n")
                    inter_id_1 = list_available_inter[
                        j - nb_og_clients - nb_synth_clients
                    ]
                    inter_loc_1 = inter_id_to_location[inter_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                ]
                            ]
                        )[0]
                    )
            else:
                new_car_matrix[i][j] = time_matrix_car[i - nb_synth_clients][
                    j - nb_synth_clients
                ]
                new_bike_matrix[i][j] = time_matrix_bike[i - nb_synth_clients][
                    j - nb_synth_clients
                ]

            new_car_matrix[j][i] = new_car_matrix[i][j]
            new_bike_matrix[j][i] = new_bike_matrix[i][j]

    for i in range(len_new_matrix):
        new_car_matrix[i][i] = 0
        new_bike_matrix[i][i] = 0

    return np.array(new_car_matrix), np.array(new_bike_matrix)

## 6.2. Add new inter


The goal is to create artificially a new inter the following way:

- Get a random ('Skills', 'Disponibility', 'Car') from existing inter (can be different ones);
- Get a random location

From the random location, infere the travel time to each other node


In [29]:
def add_clients_and_inter_in_time_matrix(
    client_synthetic_locations,
    inter_synthetic_locations,
    time_matrix_car,
    time_matrix_bike,
    df_og_need,
    list_available_inter,
):
    nb_og_clients = len(df_og_need)
    nb_og_inter = len(list_available_inter)
    len_new_matrix = (
        len(client_synthetic_locations)
        + len(inter_synthetic_locations)
        + len(time_matrix_car)
    )
    new_car_matrix = [
        [4000 for k in range(len_new_matrix)] for k in range(len_new_matrix)
    ]
    new_bike_matrix = [
        [4000 for k in range(len_new_matrix)] for k in range(len_new_matrix)
    ]

    nb_synth_clients = len(client_synthetic_locations)
    nb_synth_inter = len(inter_synthetic_locations)

    for i in range(len_new_matrix - 1):
        for j in range(i + 1, len_new_matrix):
            if i < nb_og_clients:
                if j < nb_og_clients:
                    new_car_matrix[i][j] = time_matrix_car[i][j]
                    new_bike_matrix[i][j] = time_matrix_bike[i][j]
                elif nb_og_clients <= j < nb_og_clients + nb_synth_clients:
                    client_id_1 = df_og_need["ID Client"][i]
                    client_loc_1 = client_id_to_location[client_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                elif (
                    nb_og_clients + nb_synth_clients
                    <= j
                    < nb_og_clients + nb_synth_clients + nb_og_inter
                ):
                    new_car_matrix[i][j] = time_matrix_car[i][j - nb_synth_clients]
                    new_bike_matrix[i][j] = time_matrix_bike[i][j - nb_synth_clients]
                else:
                    client_id_1 = df_og_need["ID Client"][i]
                    client_loc_1 = client_id_to_location[client_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
            elif nb_og_clients <= i < nb_og_clients + nb_synth_clients:
                if j < nb_og_clients + nb_synth_clients:
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                elif (
                    nb_og_clients + nb_synth_clients
                    <= j
                    < nb_og_clients + nb_synth_clients + nb_og_inter
                ):
                    # inter_id_1 = inter_idx_to_id[j - nb_og_clients - nb_synth_clients]
                    inter_id_1 = list_available_inter[
                        j - nb_og_clients - nb_synth_clients
                    ]
                    inter_loc_1 = inter_id_to_location[inter_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                ]
                            ]
                        )[0]
                    )
                else:
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
            elif (
                nb_og_clients + nb_synth_clients
                <= i
                < nb_og_clients + nb_synth_clients + nb_og_inter
            ):
                if j < nb_og_clients + nb_synth_clients + nb_og_inter:
                    new_car_matrix[i][j] = time_matrix_car[i - nb_synth_clients][
                        j - nb_synth_clients
                    ]
                    new_bike_matrix[i][j] = time_matrix_bike[i - nb_synth_clients][
                        j - nb_synth_clients
                    ]
                else:
                    # inter_id_1 = inter_idx_to_id[i - nb_synth_clients - nb_og_clients]
                    inter_id_1 = list_available_inter[
                        i - nb_og_clients - nb_synth_clients
                    ]
                    inter_loc_1 = inter_id_to_location[inter_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
            else:
                new_car_matrix[i][j] = round(
                    car_time_travel_estimator.predict(
                        [
                            [
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][1],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][1],
                            ]
                        ]
                    )[0]
                )
                new_bike_matrix[i][j] = round(
                    bike_time_travel_estimator.predict(
                        [
                            [
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][1],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][1],
                            ]
                        ]
                    )[0]
                )

            new_car_matrix[j][i] = new_car_matrix[i][j]
            new_bike_matrix[j][i] = new_bike_matrix[i][j]

    for i in range(len_new_matrix):
        new_car_matrix[i][i] = 0
        new_bike_matrix[i][i] = 0

    return np.array(new_car_matrix), np.array(new_bike_matrix)

In [30]:
def add_inter_in_time_matrix(
    inter_synthetic_locations,
    time_matrix_car,
    time_matrix_bike,
    df_og_need,
    list_available_inter,
):
    nb_og_clients = len(df_og_need)
    nb_og_inter = len(list_available_inter)
    len_new_matrix = +len(inter_synthetic_locations) + len(time_matrix_car)
    new_car_matrix = [
        [4000 for k in range(len_new_matrix)] for k in range(len_new_matrix)
    ]
    new_bike_matrix = [
        [4000 for k in range(len_new_matrix)] for k in range(len_new_matrix)
    ]

    for i in range(len_new_matrix - 1):
        for j in range(i + 1, len_new_matrix):
            if i < nb_og_clients:
                if j < nb_og_clients:
                    new_car_matrix[i][j] = time_matrix_car[i][j]
                    new_bike_matrix[i][j] = time_matrix_bike[i][j]
                elif nb_og_clients <= j < nb_og_clients + nb_og_inter:
                    new_car_matrix[i][j] = time_matrix_car[i][j]
                    new_bike_matrix[i][j] = time_matrix_bike[i][j]
                else:
                    # client_id_1 = client_idx_to_id[i]
                    client_id_1 = df_og_need["ID Client"][i]
                    client_loc_1 = client_id_to_location[client_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
            elif nb_og_clients <= i < nb_og_clients + nb_og_inter:
                if j < nb_og_clients + nb_og_inter:
                    new_car_matrix[i][j] = time_matrix_car[i][j]
                    new_bike_matrix[i][j] = time_matrix_bike[i][j]
                else:
                    # inter_id_1 = inter_idx_to_id[i - nb_og_clients]
                    inter_id_1 = list_available_inter[i - nb_og_clients]
                    inter_loc_1 = inter_id_to_location[inter_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
            else:
                new_car_matrix[i][j] = round(
                    car_time_travel_estimator.predict(
                        [
                            [
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_og_inter
                                ][1],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_og_inter
                                ][1],
                            ]
                        ]
                    )[0]
                )
                new_bike_matrix[i][j] = round(
                    bike_time_travel_estimator.predict(
                        [
                            [
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_og_inter
                                ][1],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_og_inter
                                ][1],
                            ]
                        ]
                    )[0]
                )

            new_car_matrix[j][i] = new_car_matrix[i][j]
            new_bike_matrix[j][i] = new_bike_matrix[i][j]

    for i in range(len_new_matrix):
        new_car_matrix[i][i] = 0
        new_bike_matrix[i][i] = 0

    return np.array(new_car_matrix), np.array(new_bike_matrix)

## 6.3. Add actors refined


In [31]:
def sample_random_loc(nb_synth_loc, list_og_locations):
    og_x_coords, og_y_coords = zip(*list_og_locations)

    # Calculate mean and standard deviation for x and y
    og_mean_x, og_std_x = np.mean(og_x_coords), np.std(og_x_coords)
    og_mean_y, og_std_y = np.mean(og_y_coords), np.std(og_y_coords)

    # Generate synthetic locations
    list_synth_locations = [
        (
            np.random.normal(og_mean_x, og_std_x),
            np.random.normal(og_mean_y, og_std_y),
        )
        for _ in range(nb_synth_loc)
    ]

    return list_synth_locations

In [32]:
def add_actors(
    df_og_needs,
    df_og_inter,
    list_available_inter,
    time_matrix_car,
    time_matrix_bike,
    nb_synth_clients=0,
    nb_synth_inter=0,
    client_locations=client_locations,
    inter_locations=inter_locations,
    inter_synth_constraints=False,
    date=None,
):
    if nb_synth_inter == 0:
        list_synth_client_loc = sample_random_loc(nb_synth_clients, client_locations)
        new_car_time_matrix, new_bike_time_matrix = add_clients_in_time_matrix(
            list_synth_client_loc,
            time_matrix_car,
            time_matrix_bike,
            df_og_needs,
            list_available_inter,
        )

        synth_client_needs_df = df_og_needs[
            ["Prestation", "Durée", "tranche_horaire"]
        ].sample(n=nb_synth_clients, replace=True)
        synth_client_needs_df.reset_index(drop=True, inplace=True)

        synth_available_inter_df = pd.DataFrame()

    elif nb_synth_clients == 0:
        synth_client_needs_df = pd.DataFrame()

        synth_inter_df = df_og_inter.sample(n=nb_synth_inter, replace=True)
        synth_inter_df.reset_index(drop=True, inplace=True)

        list_synth_inter_loc = sample_random_loc(nb_synth_inter, inter_locations)
        synth_inter_df["Latitude"] = [x[0] for x in list_synth_inter_loc]
        synth_inter_df["Longitude"] = [x[1] for x in list_synth_inter_loc]

        if inter_synth_constraints:
            condition = synth_inter_df[date] == 1
            synth_available_inter_df = synth_inter_df[condition].copy()
            nb_synth_inter = len(synth_available_inter_df)
            list_synth_available_inter_loc = [
                (
                    synth_available_inter_df["Latitude"][k],
                    synth_available_inter_df["Longitude"][k],
                )
                for k in range(len(synth_available_inter_df))
            ]
        else:
            synth_available_inter_df = synth_inter_df
            list_synth_available_inter_loc = list_synth_inter_loc

        new_car_time_matrix, new_bike_time_matrix = add_inter_in_time_matrix(
            list_synth_available_inter_loc,
            time_matrix_car,
            time_matrix_bike,
            df_og_needs,
            list_available_inter,
        )

    else:
        list_synth_client_loc = sample_random_loc(nb_synth_clients, client_locations)

        synth_client_needs_df = df_og_needs[
            ["Prestation", "Durée", "tranche_horaire"]
        ].sample(n=nb_synth_clients, replace=True)
        synth_client_needs_df.reset_index(drop=True, inplace=True)

        synth_inter_df = df_og_inter.sample(n=nb_synth_inter, replace=True)
        synth_inter_df.reset_index(drop=True, inplace=True)

        list_synth_inter_loc = sample_random_loc(nb_synth_inter, inter_locations)
        synth_inter_df["Latitude"] = [x[0] for x in list_synth_inter_loc]
        synth_inter_df["Longitude"] = [x[1] for x in list_synth_inter_loc]

        if inter_synth_constraints:
            condition = synth_inter_df[date] == 1
            synth_available_inter_df = synth_inter_df[condition].copy()
            nb_synth_inter = len(synth_available_inter_df)
            list_synth_available_inter_loc = [
                (
                    synth_available_inter_df["Latitude"][k],
                    synth_available_inter_df["Longitude"][k],
                )
                for k in range(len(synth_available_inter_df))
            ]
        else:
            synth_available_inter_df = synth_inter_df
            list_synth_available_inter_loc = list_synth_inter_loc

        new_car_time_matrix, new_bike_time_matrix = (
            add_clients_and_inter_in_time_matrix(
                list_synth_client_loc,
                list_synth_available_inter_loc,
                time_matrix_car,
                time_matrix_bike,
                df_og_needs,
                list_available_inter,
            )
        )

    return (
        new_car_time_matrix,
        new_bike_time_matrix,
        synth_client_needs_df,
        synth_available_inter_df,
    )

In [33]:
def create_real_data_model_with_synth(
    df_og_need,
    full_time_matrix_car,
    full_time_matrix_bike,
    list_available_inter,
    nb_synth_clients=0,
    nb_synth_inter=0,
    client_id_to_idx=client_id_to_idx,
    inter_id_to_idx=inter_id_to_idx,
    nb_full_client=nb_full_client,
    list_skills=list_skills,
    df_inter=df_inter,
):
    nb_needs = len(df_og_need)
    nb_inter = len(list_available_inter)
    data = {}

    # Initialization
    data["og_time_matrix_car"] = [
        [4000 for i in range(nb_needs + nb_inter)] for k in range(nb_needs + nb_inter)
    ]
    data["og_time_matrix_bike"] = [
        [4000 for i in range(nb_needs + nb_inter)] for k in range(nb_needs + nb_inter)
    ]
    data["service_times"] = [
        0 for i in range(nb_needs + nb_inter + nb_synth_clients + nb_synth_inter)
    ]
    data["time_windows"] = [
        (0, 1440)
        for i in range(nb_needs + nb_inter + nb_synth_clients + nb_synth_inter)
    ]

    data["num_vehicles"] = nb_inter + nb_synth_inter
    data["is_car"] = [
        1 for i in range(nb_inter + nb_synth_inter)
    ]  # For the moment every inter has a car
    data["starts"] = [
        nb_needs + nb_synth_clients + i for i in range(nb_inter + nb_synth_inter)
    ]
    data["ends"] = [
        nb_needs + nb_synth_clients + i for i in range(nb_inter + nb_synth_inter)
    ]

    data["vehicle_skills"] = [
        list_skills for i in range(nb_inter + nb_synth_inter)
    ]  # For the moment every inter has all skills possible
    data["node_requirements"] = [
        None for i in range(nb_needs + nb_inter + nb_synth_clients + nb_synth_inter)
    ]

    # Create original time matrix
    for i in range(nb_needs):
        need_id_client = df_og_need["ID Client"][i]
        need_idx_client = client_id_to_idx[need_id_client]

        for j in range(nb_needs):
            other_id_client = df_og_need["ID Client"][j]
            other_idx_client = client_id_to_idx[other_id_client]

            travel_time_car = full_time_matrix_car[need_idx_client, other_idx_client]
            travel_time_bike = full_time_matrix_bike[need_idx_client, other_idx_client]
            data["og_time_matrix_car"][i][j] = travel_time_car
            data["og_time_matrix_bike"][i][j] = travel_time_bike

        for j in range(nb_inter):
            id_inter = list_available_inter[j]
            idx_inter = inter_id_to_idx[id_inter]

            travel_time_car = full_time_matrix_car[need_idx_client][
                nb_full_client + idx_inter
            ]
            travel_time_bike = full_time_matrix_bike[need_idx_client][
                nb_full_client + idx_inter
            ]

            data["og_time_matrix_car"][i][nb_needs + j] = travel_time_car
            data["og_time_matrix_car"][nb_needs + j][i] = data["og_time_matrix_car"][i][
                nb_needs + j
            ]
            data["og_time_matrix_bike"][i][nb_needs + j] = travel_time_bike
            data["og_time_matrix_bike"][nb_needs + j][i] = data["og_time_matrix_bike"][
                i
            ][nb_needs + j]
    for i in range(nb_needs + nb_inter):
        data["og_time_matrix_car"][i][i] = 0
        data["og_time_matrix_bike"][i][i] = 0

    # Create new time matrix from orginial adding synthetic travel times
    (
        data["time_matrix_car"],
        data["time_matrix_bike"],
        synth_client_needs_df,
        synth_inter_df,
    ) = add_actors(
        df_og_need,
        df_inter,
        list_available_inter,
        data["og_time_matrix_car"],
        data["og_time_matrix_bike"],
        nb_synth_clients,
        nb_synth_inter,
        client_locations,
        inter_locations,
        inter_synth_constraints=int,
    )

    for i in range(nb_needs + nb_synth_clients):
        if i < nb_needs:
            service_time = int(df_og_need["Durée"][i])
            data["service_times"][i] = service_time

            time_window_temp = (
                df_og_need["tranche_horaire"][i]
                .replace("[", "")
                .replace("]", "")
                .replace(",", "")
                .split()
            )
            time_window_start = int(float(time_window_temp[0]) * 60)
            time_window_end = int(float(time_window_temp[1]) * 60)
            data["time_windows"][i] = (time_window_start, time_window_end)

            skills_required = df_og_need["Prestation"][i]
            data["node_requirements"][i] = skills_required
        else:
            service_time = int(synth_client_needs_df["Durée"][i - nb_needs])
            data["service_times"][i] = service_time

            time_window_temp = (
                synth_client_needs_df["tranche_horaire"][i - nb_needs]
                .replace("[", "")
                .replace("]", "")
                .replace(",", "")
                .split()
            )
            time_window_start = int(float(time_window_temp[0]) * 60)
            time_window_end = int(float(time_window_temp[1]) * 60)
            data["time_windows"][i] = (time_window_start, time_window_end)

            skills_required = synth_client_needs_df["Prestation"][i - nb_needs]
            data["node_requirements"][i] = skills_required

    data["time_windows_with_service"] = [
        (
            data["time_windows"][i][0] + data["service_times"][i],
            data["time_windows"][i][1] + data["service_times"][i],
        )
        for i in range(len(data["service_times"]))
    ]

    data["workload_per_node"] = [1 for i in range(nb_needs + nb_synth_clients)].append(
        [0 for k in range(nb_inter + nb_synth_inter)]
    )  # Useless for now but could be used to differ workload from each node
    data["workload_capacity"] = [5 for i in range(nb_inter + nb_synth_inter)]

    return data

## 6.4. Refined printing result function


In [34]:
def eq_co2_car(travel_time):
    # 10km = 1.9kg CO2
    speed = 0.5  # km/min
    return 1.9 * travel_time * speed / 10


def eq_co2_elec_bike(travel_time):
    # 10km = 20g CO2
    speed = 1 / 3  # km/min
    return 0.02 * travel_time * speed / 10

In [149]:
def print_detailed_solution_real_data(
    data, manager, routing, solution, list_available_inter, verbose=0
):
    """Prints detailed solution on console, including service and wait times."""
    # print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")

    nb_cars_used = 0
    nb_bikes_used = 0
    nb_inter_used = 0

    nb_break_inf = 0
    nb_break_sup = 0
    nb_visits = 0

    avg_length_day = 0
    list_length_day = []

    overall_total_car_commuting_time = 0
    overall_total_bike_commuting_time = 0
    overall_total_commuting_time = 0  # Initialize overall commuting time

    # Display dropped nodes.
    dropped_nodes = "Dropped nodes:"
    count_dropped_nodes = 0
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if solution.Value(routing.NextVar(node)) == node:
            dropped_nodes += f" {manager.IndexToNode(node)};"
            count_dropped_nodes += 1
    print(f"{dropped_nodes} (Total: {count_dropped_nodes})")

    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        vehicle_skills = str(data["vehicle_skills"][vehicle_id])
        try:
            inter_id = list_available_inter[vehicle_id]
        except IndexError:
            inter_id = "*synthetic*"
        vehicle_type_str = "Car" if data["is_car"][vehicle_id] else "Bike"
        if verbose == 1:
            print(
                f"Route for vehicle {inter_id} (Type: {vehicle_type_str}; Skills: {vehicle_skills}):"
            )

        vehicle_commuting_time = 0  # Initialize commuting time for the current vehicle
        nb_steps = 0
        start_day = 0
        end_day = 0
        nb_break_inf_veh = 0
        nb_break_sup_veh = 0

        # print(f"Route for vehicle {inter_id}:")
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            next_index = solution.Value(routing.NextVar(index))
            next_node = manager.IndexToNode(next_index)

            if nb_steps == 1:
                if data["is_car"][vehicle_id]:
                    nb_cars_used += 1
                else:
                    nb_bikes_used += 1
                nb_inter_used += 1

            service_time = data["service_times"][manager.IndexToNode(index)]
            skills_required = data["node_requirements"][manager.IndexToNode(index)]
            # Arrival time at current location
            arrival_time = solution.Min(time_var) - service_time

            # End of service time = arrival time + service time at current location
            end_of_service_time = solution.Min(time_var)

            # Initialization
            departure_time = end_of_service_time

            if data["is_car"][vehicle_id]:
                next_travel_time = data["time_matrix_car"][manager.IndexToNode(index)][
                    next_node
                ]
            else:
                next_travel_time = data["time_matrix_bike"][manager.IndexToNode(index)][
                    next_node
                ]
            vehicle_commuting_time += next_travel_time
            if data["is_car"][vehicle_id]:
                overall_total_car_commuting_time += next_travel_time
            else:
                overall_total_bike_commuting_time += next_travel_time
            overall_total_commuting_time += next_travel_time

            if next_index != routing.End(vehicle_id):
                next_end_of_service_time = solution.Min(
                    time_dimension.CumulVar(next_index)
                )
                next_service_time = data["service_times"][next_node]
                next_arrival_time = next_end_of_service_time - next_service_time
                departure_time = next_arrival_time - next_travel_time

            break_time = departure_time - end_of_service_time

            if nb_steps > 0:
                if break_time > 30:
                    nb_break_sup += 1
                    nb_break_sup_veh += 1
                    nb_visits += 1
                else:
                    nb_visits += 1
                    if next_index != routing.End(vehicle_id):
                        nb_break_inf += 1
                        nb_break_inf_veh += 1

            if nb_steps == 0:
                start_day = departure_time

            nb_steps += 1

            if next_index == routing.End(vehicle_id):
                break_time = 0
                departure_time = end_of_service_time

            arrival_time_str = (
                str(arrival_time // 60)
                + "h"
                + (
                    str(arrival_time % 60)
                    if arrival_time % 60 >= 10
                    else "0" + str(arrival_time % 60)
                )
            )
            end_of_service_time_str = (
                str(end_of_service_time // 60)
                + "h"
                + (
                    str(end_of_service_time % 60)
                    if end_of_service_time % 60 >= 10
                    else "0" + str(end_of_service_time % 60)
                )
            )
            break_time_str = (
                str(break_time // 60)
                + "h"
                + (
                    str(break_time % 60)
                    if break_time % 60 >= 10
                    else "0" + str(break_time % 60)
                )
            )
            departure_time_str = (
                str(departure_time // 60)
                + "h"
                + (
                    str(departure_time % 60)
                    if departure_time % 60 >= 10
                    else "0" + str(departure_time % 60)
                )
            )

            if verbose == 1:
                # Print detailed timing for current location
                print(
                    f"* Location {manager.IndexToNode(index)} (Arrival: {arrival_time_str}, "
                    f"End of Service Time (Service time: {service_time}) (Skills required: {skills_required}): {end_of_service_time_str}, Break: {break_time_str}, "
                    f"Departure: {departure_time_str}, Next travel time: {next_travel_time}) -> "
                )

            index = next_index

        # Handle final location's arrival time
        final_arrival_time = solution.Min(time_dimension.CumulVar(index))
        end_day = final_arrival_time
        final_arrival_time_str = (
            str(final_arrival_time // 60)
            + "h"
            + (
                str(final_arrival_time % 60)
                if final_arrival_time % 60 >= 10
                else "0" + str(final_arrival_time % 60)
            )
        )
        if verbose == 1:
            print(
                f"* Location {manager.IndexToNode(index) + 1} (Arrival: {final_arrival_time_str})"
            )
            print(
                f"Number of small breaks (<30min): {nb_break_inf_veh}; Number of big breaks (>30min): {nb_break_sup_veh}"
            )
            print(
                f"Total commuting time for vehicle {inter_id}: {vehicle_commuting_time} minutes"
            )

        if end_day - start_day > 1:  # Superior than 1 minute
            list_length_day.append((start_day, end_day))
            avg_length_day += end_day - start_day

            start_day_str = (
                str(start_day // 60)
                + "h"
                + (
                    str(start_day % 60)
                    if start_day % 60 >= 10
                    else "0" + str(start_day % 60)
                )
            )
            end_day_str = (
                str(end_day // 60)
                + "h"
                + (str(end_day % 60) if end_day % 60 >= 10 else "0" + str(end_day % 60))
            )

            if verbose == 1:
                print(
                    f"Start of the day: {start_day_str}; End of the day: {end_day_str}"
                )
        if verbose == 1:
            print("\n")

    nb_available_inter = data["num_vehicles"]
    print(f"Total available inter: {nb_available_inter}")
    print(f"Total inter who worked: {nb_inter_used}")
    print(f"Toal number of visited nodes: {nb_visits}")
    print(f"Overall total commuting time: {overall_total_commuting_time} minutes")
    print(f"Number of cars used: {nb_cars_used}")
    eq_co2_cars = eq_co2_car(overall_total_car_commuting_time)
    print(
        f"Overall total commuting time for cars: {overall_total_car_commuting_time} minutes ({eq_co2_cars} kgCO2)"
    )
    print(f"Number of bikes used: {nb_bikes_used}")
    eq_co2_bikes = eq_co2_elec_bike(overall_total_bike_commuting_time)
    print(
        f"Overall total commuting time for bikes: {overall_total_bike_commuting_time} minutes ({eq_co2_bikes} kgCO2)"
    )
    print(
        f"Total number of small breaks (<30min): {nb_break_inf}; Total number of big breaks (>30min): {nb_break_sup}"
    )

    avg_length_day = avg_length_day / nb_inter_used
    avg_length_day_str = round(avg_length_day)
    avg_length_day_str = (
        str(avg_length_day_str // 60)
        + "h"
        + (
            str(avg_length_day_str % 60)
            if avg_length_day_str % 60 >= 10
            else "0" + str(avg_length_day_str % 60)
        )
    )
    print(f"Average length of a day of work: {avg_length_day_str}")

    dict_results = {
        "total_commuting_time": overall_total_commuting_time,
        "total_car_commuting_time": overall_total_car_commuting_time,
        "total_bike_commuting_time": overall_total_bike_commuting_time,
        "nb_available_inter": nb_available_inter,
        "nb_inter_used": nb_inter_used,
        "nb_cars_used": nb_cars_used,
        "nb_bikes_used": nb_bikes_used,
        "avg_length_day": avg_length_day,
        "list_length_day": list_length_day,
        "eq_co2_cars": eq_co2_cars,
        "eq_co2_bikes": eq_co2_bikes,
        "nb_break_inf": nb_break_inf,
        "nb_break_sup": nb_break_sup,
        "nb_visits": nb_visits,
        "nb_dropped_nodes": count_dropped_nodes,
    }

    return dict_results

## 6.5. Bikes fleet proportion


In [114]:
def get_is_car(list_available_inter, df_inter):
    list_is_car = []
    for id_inter in list_available_inter:
        condition = df_inter["ID Intervenant"] == id_inter
        if df_inter.loc[condition, "Véhicule personnel"].values[0] == "Oui":
            list_is_car.append(1)
        else:
            list_is_car.append(0)

    return list_is_car

In [115]:
def get_synth_is_car(df_synth_inter):
    list_is_car = []
    for car in df_synth_inter["Véhicule personnel"]:
        if car == "Oui":
            list_is_car.append(1)
        else:
            list_is_car.append(0)

    return list_is_car

Adding the 'bike_proportion' parameter


In [116]:
import random

In [117]:
def generate_vehicle_type_list(nb_inter, bike_proportion):
    nb_bikes = int(nb_inter * bike_proportion)
    vehicle_type_list = [0] * nb_bikes + [1] * (nb_inter - nb_bikes)
    random.shuffle(vehicle_type_list)
    return vehicle_type_list

## 6.6 Inter constraints


In [118]:
def get_inter_skills(list_available_inter, df_inter):
    list_all_inter_skills = []
    for id_inter in list_available_inter:
        condition = df_inter["ID Intervenant"] == id_inter
        tmp_skills = df_inter.loc[condition, "Compétences"].values[0]
        list_skills = [x.strip() for x in tmp_skills.split(sep=",")]
        list_all_inter_skills.append(list_skills)

    return list_all_inter_skills

In [119]:
def get_synth_inter_skills(synth_inter_df):
    list_all_inter_skills = []
    for tmp_skills in synth_inter_df["Compétences"]:
        list_skills = [x.strip() for x in tmp_skills.split(sep=",")]
        list_all_inter_skills.append(list_skills)

    return list_all_inter_skills

In [145]:
# 1: Availability constraints
def create_real_data_model_with_synth(
    df_og_need,
    full_time_matrix_car,
    full_time_matrix_bike,
    list_available_inter,
    nb_synth_clients=0,
    nb_synth_inter=0,
    client_id_to_idx=client_id_to_idx,
    inter_id_to_idx=inter_id_to_idx,
    nb_full_client=nb_full_client,
    list_skills=list_skills,
    df_inter=df_inter,
    inter_constraints=False,
    date=None,
    bike_proportion=0.0,
    workload_capacity=7,
):
    nb_needs = len(df_og_need)
    nb_inter = len(list_available_inter)
    data = {}

    # Initialization
    data["og_time_matrix_car"] = [
        [4000 for i in range(nb_needs + nb_inter)] for k in range(nb_needs + nb_inter)
    ]
    data["og_time_matrix_bike"] = [
        [4000 for i in range(nb_needs + nb_inter)] for k in range(nb_needs + nb_inter)
    ]

    # Create original time matrix
    for i in range(nb_needs):
        need_id_client = df_og_need["ID Client"][i]
        need_idx_client = client_id_to_idx[need_id_client]

        for j in range(nb_needs):
            other_id_client = df_og_need["ID Client"][j]
            other_idx_client = client_id_to_idx[other_id_client]

            travel_time_car = full_time_matrix_car[need_idx_client, other_idx_client]
            travel_time_bike = full_time_matrix_bike[need_idx_client, other_idx_client]
            data["og_time_matrix_car"][i][j] = travel_time_car
            data["og_time_matrix_bike"][i][j] = travel_time_bike

        for j in range(nb_inter):
            id_inter = list_available_inter[j]
            idx_inter = inter_id_to_idx[id_inter]

            travel_time_car = full_time_matrix_car[need_idx_client][
                nb_full_client + idx_inter
            ]
            travel_time_bike = full_time_matrix_bike[need_idx_client][
                nb_full_client + idx_inter
            ]

            data["og_time_matrix_car"][i][nb_needs + j] = travel_time_car
            data["og_time_matrix_car"][nb_needs + j][i] = data["og_time_matrix_car"][i][
                nb_needs + j
            ]
            data["og_time_matrix_bike"][i][nb_needs + j] = travel_time_bike
            data["og_time_matrix_bike"][nb_needs + j][i] = data["og_time_matrix_bike"][
                i
            ][nb_needs + j]
    for i in range(nb_needs + nb_inter):
        data["og_time_matrix_car"][i][i] = 0
        data["og_time_matrix_bike"][i][i] = 0

    # Create new time matrix from orginial adding synthetic travel times
    (
        data["time_matrix_car"],
        data["time_matrix_bike"],
        synth_client_needs_df,
        synth_available_inter_df,
    ) = add_actors(
        df_og_need,
        df_inter,
        list_available_inter,
        data["og_time_matrix_car"],
        data["og_time_matrix_bike"],
        nb_synth_clients,
        nb_synth_inter,
        client_locations,
        inter_locations,
        inter_synth_constraints=inter_constraints,
        date=date,
    )

    if inter_constraints:
        nb_synth_inter = len(synth_available_inter_df)

    data["service_times"] = [
        0 for i in range(nb_needs + nb_inter + nb_synth_clients + nb_synth_inter)
    ]
    data["time_windows"] = [
        (0, 1440)
        for i in range(nb_needs + nb_inter + nb_synth_clients + nb_synth_inter)
    ]

    data["num_vehicles"] = nb_inter + nb_synth_inter

    if inter_constraints:
        list_is_car = get_is_car(list_available_inter, df_inter)
        if nb_synth_inter > 0:
            list_is_car = [*list_is_car, *get_synth_is_car(synth_available_inter_df)]
        data["is_car"] = list_is_car
    else:
        data["is_car"] = [1 for i in range(nb_inter + nb_synth_inter)]
    data["starts"] = [
        nb_needs + nb_synth_clients + i for i in range(nb_inter + nb_synth_inter)
    ]
    data["ends"] = [
        nb_needs + nb_synth_clients + i for i in range(nb_inter + nb_synth_inter)
    ]

    if inter_constraints:
        list_inter_skills = get_inter_skills(list_available_inter, df_inter)
        if nb_synth_inter > 0:
            list_inter_skills = [
                *list_inter_skills,
                *get_synth_inter_skills(synth_available_inter_df),
            ]
        data["vehicle_skills"] = list_inter_skills
    else:
        data["vehicle_skills"] = [list_skills for i in range(nb_inter + nb_synth_inter)]
    data["node_requirements"] = [
        None for i in range(nb_needs + nb_inter + nb_synth_clients + nb_synth_inter)
    ]

    for i in range(nb_needs + nb_synth_clients):
        if i < nb_needs:
            service_time = int(df_og_need["Durée"][i])
            data["service_times"][i] = service_time

            time_window_temp = (
                df_og_need["tranche_horaire"][i]
                .replace("[", "")
                .replace("]", "")
                .replace(",", "")
                .split()
            )
            time_window_start = int(float(time_window_temp[0]) * 60)
            time_window_end = int(float(time_window_temp[1]) * 60)
            data["time_windows"][i] = (time_window_start, time_window_end)

            skills_required = df_og_need["Prestation"][i]
            data["node_requirements"][i] = skills_required
        else:
            service_time = int(synth_client_needs_df["Durée"][i - nb_needs])
            data["service_times"][i] = service_time

            time_window_temp = (
                synth_client_needs_df["tranche_horaire"][i - nb_needs]
                .replace("[", "")
                .replace("]", "")
                .replace(",", "")
                .split()
            )
            time_window_start = int(float(time_window_temp[0]) * 60)
            time_window_end = int(float(time_window_temp[1]) * 60)
            data["time_windows"][i] = (time_window_start, time_window_end)

            skills_required = synth_client_needs_df["Prestation"][i - nb_needs]
            data["node_requirements"][i] = skills_required

    data["time_windows_with_service"] = [
        (
            data["time_windows"][i][0] + data["service_times"][i],
            data["time_windows"][i][1] + data["service_times"][i],
        )
        for i in range(len(data["service_times"]))
    ]

    data["workload_per_node"] = [
        *[1 for i in range(nb_needs + nb_synth_clients)],
        *[0 for k in range(nb_inter + nb_synth_inter)],
    ]  # Useless for now but could be used to differ workload from each node
    data["workload_capacity"] = [
        workload_capacity for i in range(nb_inter + nb_synth_inter)
    ]

    if bike_proportion > 0.00001:
        data["is_car"] = generate_vehicle_type_list(
            nb_synth_inter + nb_inter, bike_proportion
        )

    return data

## 6.7. Training policies


Add training for inter


In [ ]:
# 1: Mandatory

# 7. Running scenarios


In [121]:
list_dates = [
    "2024-01-15",
    "2024-01-16",
    "2024-01-17",
    "2024-01-18",
    "2024-01-19",
    "2024-01-20",
    "2024-01-21",
]
list_skills = c_l_need["Prestation"].unique()

In [134]:
def main_real_data_with_synth(
    df_need,
    full_time_matrix_car,
    full_time_matrix_bike,
    list_available_inter,
    nb_synth_clients=0,
    nb_synth_inter=0,
    workload_capacity=7,
    bike_proportion=0,
    verbose=0,
    inter_constraints=False,
    date=None,
):
    """Solve the VRP with time windows from specified locations."""
    # Instantiate the data problem.
    data = create_real_data_model_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        list_available_inter,
        nb_synth_clients=nb_synth_clients,
        nb_synth_inter=nb_synth_inter,
        bike_proportion=bike_proportion,
        workload_capacity=workload_capacity
        + 1,  # To count the initial double travel (from home / to home)
        inter_constraints=inter_constraints,
        date=date,
    )

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["time_matrix_car"]), data["num_vehicles"], data["starts"], data["ends"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def time_callback(from_index, to_index, type):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        service_time = data["service_times"][to_node]
        if type == 1:  # Car
            return data["time_matrix_car"][from_node][to_node] + service_time
            # return data["time_matrix_car"][from_node][to_node]
        else:
            return data["time_matrix_bike"][from_node][to_node] + service_time
            # return data["time_matrix_bike"][from_node][to_node]

    car_callback = partial(time_callback, type=1)
    bike_callback = partial(time_callback, type=0)

    car_index = routing.RegisterTransitCallback(car_callback)
    bike_index = routing.RegisterTransitCallback(bike_callback)

    list_index = []
    for k in range(data["num_vehicles"]):
        if data["is_car"][k] == 1:
            routing.SetArcCostEvaluatorOfVehicle(car_index, k)
            list_index.append(car_index)
        else:
            routing.SetArcCostEvaluatorOfVehicle(bike_index, k)
            list_index.append(bike_index)

    # Enforce node requirements based on vehicle skills
    for node_idx, requirement in enumerate(data["node_requirements"]):
        if requirement:  # Check if the node has specific skill requirements
            allowed_vehicles = [
                vehicle_id
                for vehicle_id, skills in enumerate(data["vehicle_skills"])
                if requirement in skills
            ]
            manager_idx = manager.NodeToIndex(node_idx)
            if manager_idx >= 0:  # Ensure manager index is valid
                routing.SetAllowedVehiclesForIndex(allowed_vehicles, manager_idx)

    # Add Time constraint.
    dimension_name = "Time"
    routing.AddDimensionWithVehicleTransitAndCapacity(
        list_index,
        10000,  # no slack
        [10000 for i in range(data["num_vehicles"])],  # vehicle maximum travel time
        True,  # start cumul to zero
        dimension_name,
    )

    time_dimension = routing.GetDimensionOrDie(dimension_name)

    def workload_callback(from_index):
        return 1

    workload_index = routing.RegisterUnaryTransitCallback(workload_callback)
    routing.AddDimensionWithVehicleCapacity(
        workload_index,
        slack_max=0,  # No slack
        vehicle_capacities=data["workload_capacity"],  # Maximum "load" per vehicle
        fix_start_cumul_to_zero=True,
        name="LoadBalance",
    )

    # Allow to drop nodes.
    penalty = 10000
    nb_needs = len(df_need) + nb_synth_clients
    for node in range(nb_needs):
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

    # Add time window constraints for each location except start and end.
    for location_idx, time_window in enumerate(data["time_windows_with_service"]):
        if location_idx in data["starts"]:
            continue
        if location_idx in data["ends"]:
            continue
        index = manager.NodeToIndex(location_idx)
        start_time_window = time_window[0]
        end_time_window = time_window[1]
        time_dimension.CumulVar(index).SetRange(start_time_window, end_time_window)

    # Instantiate route start and end times to produce feasible times.
    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i))
        )
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    # search_parameters.log_search = True
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        # print_solution(data, manager, routing, solution)
        dict_result = print_detailed_solution_real_data(
            data, manager, routing, solution, list_available_inter, verbose=verbose
        )
        dict_result["data"] = data
        return dict_result

## 7.1.A. Pas de contraintes


In [150]:
list_scenario_7_1_a = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")

    dict_results = main_real_data_with_synth(
        df_need, full_time_matrix_car, full_time_matrix_bike, full_inter_list, verbose=0
    )

    print("\n\n\n")

    list_scenario_7_1_a.append(dict_results)

Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 18
Toal number of visited nodes: 103
Overall total commuting time: 297 minutes
Number of cars used: 18
Overall total commuting time for cars: 297 minutes (28.214999999999996 kgCO2)
Number of bikes used: 0
Overall total commuting time for bikes: 0 minutes (0.0 kgCO2)
Total number of small breaks (<30min): 63; Total number of big breaks (>30min): 22
Average length of a day of work: 10h26




Date: 2024-01-16
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 20
Toal number of visited nodes: 111
Overall total commuting time: 385 minutes
Number of cars used: 20
Overall total commuting time for cars: 385 minutes (36.575 kgCO2)
Number of bikes used: 0
Overall total commuting time for bikes: 0 minutes (0.0 kgCO2)
Total number of small breaks (<30min): 60; Total number of big breaks (>30min): 31
Average length of a day of work: 10h51




Date: 2024-01-17
Dropped nodes: (Total: 

## 7.1.B. Avec contraintes


In [151]:
list_scenario_7_1_b = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")
    list_available_inter = get_list_og_available(date, df_inter)

    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        list_available_inter,
        inter_constraints=True,
        workload_capacity=7,
        verbose=0,
        date=date,
    )

    print("\n\n\n")

    list_scenario_7_1_b.append(dict_results)

Date: 2024-01-15
Dropped nodes: 67; (Total: 1)
Total available inter: 17
Total inter who worked: 16
Toal number of visited nodes: 102
Overall total commuting time: 575 minutes
Number of cars used: 13
Overall total commuting time for cars: 410 minutes (38.95 kgCO2)
Number of bikes used: 3
Overall total commuting time for bikes: 165 minutes (0.11000000000000001 kgCO2)
Total number of small breaks (<30min): 63; Total number of big breaks (>30min): 23
Average length of a day of work: 10h51




Date: 2024-01-16
Dropped nodes: 17; 88; (Total: 2)
Total available inter: 18
Total inter who worked: 18
Toal number of visited nodes: 109
Overall total commuting time: 721 minutes
Number of cars used: 14
Overall total commuting time for cars: 631 minutes (59.94499999999999 kgCO2)
Number of bikes used: 4
Overall total commuting time for bikes: 90 minutes (0.06 kgCO2)
Total number of small breaks (<30min): 73; Total number of big breaks (>30min): 18
Average length of a day of work: 9h54




Date: 2024-

## 7.2. Proportions de vélo (tous les intervenants disponibles, tous les skills, pas de clients en plus)


### 7.2.A. Proportion: 0%


In [152]:
list_scenario_7_2_a = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        full_inter_list,
        inter_constraints=False,
        bike_proportion=0.0,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_2_a.append(dict_results)

Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 18
Toal number of visited nodes: 103
Overall total commuting time: 281 minutes
Number of cars used: 18
Overall total commuting time for cars: 281 minutes (26.695 kgCO2)
Number of bikes used: 0
Overall total commuting time for bikes: 0 minutes (0.0 kgCO2)
Total number of small breaks (<30min): 63; Total number of big breaks (>30min): 22
Average length of a day of work: 10h40




Date: 2024-01-16
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 20
Toal number of visited nodes: 111
Overall total commuting time: 380 minutes
Number of cars used: 20
Overall total commuting time for cars: 380 minutes (36.1 kgCO2)
Number of bikes used: 0
Overall total commuting time for bikes: 0 minutes (0.0 kgCO2)
Total number of small breaks (<30min): 61; Total number of big breaks (>30min): 30
Average length of a day of work: 10h58




Date: 2024-01-17
Dropped nodes: (Total: 0)
Total avail

In [156]:
list_scenario_7_2_a_bis = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")
    list_available_inter = get_list_og_available(date, df_inter)

    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        list_available_inter,
        inter_constraints=True,
        workload_capacity=7,
        verbose=0,
        date=date,
        bike_proportion=0.0,
    )

    print("\n\n\n")

    list_scenario_7_2_a_bis.append(dict_results)

Date: 2024-01-15
Dropped nodes: 67; (Total: 1)
Total available inter: 17
Total inter who worked: 16
Toal number of visited nodes: 102
Overall total commuting time: 568 minutes
Number of cars used: 13
Overall total commuting time for cars: 403 minutes (38.285 kgCO2)
Number of bikes used: 3
Overall total commuting time for bikes: 165 minutes (0.11000000000000001 kgCO2)
Total number of small breaks (<30min): 63; Total number of big breaks (>30min): 23
Average length of a day of work: 10h50




Date: 2024-01-16
Dropped nodes: 17; (Total: 1)
Total available inter: 18
Total inter who worked: 18
Toal number of visited nodes: 110
Overall total commuting time: 615 minutes
Number of cars used: 14
Overall total commuting time for cars: 530 minutes (50.35 kgCO2)
Number of bikes used: 4
Overall total commuting time for bikes: 85 minutes (0.056666666666666664 kgCO2)
Total number of small breaks (<30min): 70; Total number of big breaks (>30min): 22
Average length of a day of work: 10h25




Date: 202

### 7.2.B. Proportion: 10%


In [153]:
list_scenario_7_2_b = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        full_inter_list,
        inter_constraints=False,
        bike_proportion=0.1,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_2_b.append(dict_results)

Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 18
Toal number of visited nodes: 103
Overall total commuting time: 330 minutes
Number of cars used: 18
Overall total commuting time for cars: 330 minutes (31.35 kgCO2)
Number of bikes used: 0
Overall total commuting time for bikes: 0 minutes (0.0 kgCO2)
Total number of small breaks (<30min): 59; Total number of big breaks (>30min): 26
Average length of a day of work: 10h59




Date: 2024-01-16
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 19
Toal number of visited nodes: 111
Overall total commuting time: 430 minutes
Number of cars used: 18
Overall total commuting time for cars: 424 minutes (40.279999999999994 kgCO2)
Number of bikes used: 1
Overall total commuting time for bikes: 6 minutes (0.003999999999999999 kgCO2)
Total number of small breaks (<30min): 62; Total number of big breaks (>30min): 30
Average length of a day of work: 11h06




Date: 2024-01-17
Dropped

In [157]:
list_scenario_7_2_b_bis = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")
    list_available_inter = get_list_og_available(date, df_inter)

    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        list_available_inter,
        inter_constraints=True,
        workload_capacity=7,
        verbose=0,
        date=date,
        bike_proportion=0.1,
    )

    print("\n\n\n")

    list_scenario_7_2_b_bis.append(dict_results)

Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 17
Total inter who worked: 16
Toal number of visited nodes: 103
Overall total commuting time: 496 minutes
Number of cars used: 15
Overall total commuting time for cars: 484 minutes (45.98 kgCO2)
Number of bikes used: 1
Overall total commuting time for bikes: 12 minutes (0.007999999999999998 kgCO2)
Total number of small breaks (<30min): 68; Total number of big breaks (>30min): 19
Average length of a day of work: 10h47




Date: 2024-01-16
Dropped nodes: (Total: 0)
Total available inter: 18
Total inter who worked: 17
Toal number of visited nodes: 111
Overall total commuting time: 652 minutes
Number of cars used: 17
Overall total commuting time for cars: 652 minutes (61.94 kgCO2)
Number of bikes used: 0
Overall total commuting time for bikes: 0 minutes (0.0 kgCO2)
Total number of small breaks (<30min): 72; Total number of big breaks (>30min): 22
Average length of a day of work: 10h46




Date: 2024-01-17
Dropped nodes: (Tot

### 7.2.C. Proportion: 25%


In [154]:
list_scenario_7_2_c = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        full_inter_list,
        inter_constraints=False,
        bike_proportion=0.25,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_2_c.append(dict_results)

Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 20
Toal number of visited nodes: 103
Overall total commuting time: 323 minutes
Number of cars used: 18
Overall total commuting time for cars: 309 minutes (29.355 kgCO2)
Number of bikes used: 2
Overall total commuting time for bikes: 14 minutes (0.009333333333333334 kgCO2)
Total number of small breaks (<30min): 54; Total number of big breaks (>30min): 29
Average length of a day of work: 10h24




Date: 2024-01-16
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 20
Toal number of visited nodes: 111
Overall total commuting time: 407 minutes
Number of cars used: 18
Overall total commuting time for cars: 385 minutes (36.575 kgCO2)
Number of bikes used: 2
Overall total commuting time for bikes: 22 minutes (0.014666666666666666 kgCO2)
Total number of small breaks (<30min): 64; Total number of big breaks (>30min): 27
Average length of a day of work: 11h00




Date: 2024-01-17

In [158]:
list_scenario_7_2_c_bis = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")
    list_available_inter = get_list_og_available(date, df_inter)

    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        list_available_inter,
        inter_constraints=True,
        workload_capacity=7,
        verbose=0,
        date=date,
        bike_proportion=0.25,
    )

    print("\n\n\n")

    list_scenario_7_2_c_bis.append(dict_results)

Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 17
Total inter who worked: 17
Toal number of visited nodes: 103
Overall total commuting time: 611 minutes
Number of cars used: 13
Overall total commuting time for cars: 456 minutes (43.32 kgCO2)
Number of bikes used: 4
Overall total commuting time for bikes: 155 minutes (0.10333333333333332 kgCO2)
Total number of small breaks (<30min): 64; Total number of big breaks (>30min): 22
Average length of a day of work: 10h20




Date: 2024-01-16
Dropped nodes: 17; 74; 88; (Total: 3)
Total available inter: 18
Total inter who worked: 18
Toal number of visited nodes: 108
Overall total commuting time: 685 minutes
Number of cars used: 14
Overall total commuting time for cars: 550 minutes (52.25 kgCO2)
Number of bikes used: 4
Overall total commuting time for bikes: 135 minutes (0.09 kgCO2)
Total number of small breaks (<30min): 68; Total number of big breaks (>30min): 22
Average length of a day of work: 10h38




Date: 2024-01-17
Drop

### 7.2.D. Proportion: 50%


In [155]:
list_scenario_7_2_d = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        full_inter_list,
        inter_constraints=False,
        bike_proportion=0.5,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_2_d.append(dict_results)

Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 18
Toal number of visited nodes: 103
Overall total commuting time: 355 minutes
Number of cars used: 12
Overall total commuting time for cars: 269 minutes (25.555 kgCO2)
Number of bikes used: 6
Overall total commuting time for bikes: 86 minutes (0.057333333333333326 kgCO2)
Total number of small breaks (<30min): 64; Total number of big breaks (>30min): 21
Average length of a day of work: 9h59




Date: 2024-01-16
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 20
Toal number of visited nodes: 111
Overall total commuting time: 528 minutes
Number of cars used: 12
Overall total commuting time for cars: 415 minutes (39.425 kgCO2)
Number of bikes used: 8
Overall total commuting time for bikes: 113 minutes (0.07533333333333334 kgCO2)
Total number of small breaks (<30min): 68; Total number of big breaks (>30min): 23
Average length of a day of work: 9h51




Date: 2024-01-17
D

In [159]:
list_scenario_7_2_d_bis = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")
    list_available_inter = get_list_og_available(date, df_inter)

    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        list_available_inter,
        inter_constraints=True,
        workload_capacity=7,
        verbose=0,
        date=date,
        bike_proportion=0.5,
    )

    print("\n\n\n")

    list_scenario_7_2_d_bis.append(dict_results)

Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 17
Total inter who worked: 17
Toal number of visited nodes: 103
Overall total commuting time: 652 minutes
Number of cars used: 9
Overall total commuting time for cars: 281 minutes (26.695 kgCO2)
Number of bikes used: 8
Overall total commuting time for bikes: 371 minutes (0.24733333333333332 kgCO2)
Total number of small breaks (<30min): 63; Total number of big breaks (>30min): 23
Average length of a day of work: 10h43




Date: 2024-01-16
Dropped nodes: 17; 88; (Total: 2)
Total available inter: 18
Total inter who worked: 17
Toal number of visited nodes: 109
Overall total commuting time: 687 minutes
Number of cars used: 9
Overall total commuting time for cars: 381 minutes (36.195 kgCO2)
Number of bikes used: 8
Overall total commuting time for bikes: 306 minutes (0.20400000000000001 kgCO2)
Total number of small breaks (<30min): 68; Total number of big breaks (>30min): 24
Average length of a day of work: 11h05




Date: 2024

## 7.3. Ajout de compétences
